In [1]:
import pandas as pd
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

/home/shilpi/Documents/sem3/P2_BiasMitigation_DataAugmentation/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the LLaMA model
model_id = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map='auto', torch_dtype=torch.float16)

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.43s/it]


In [3]:
# Load the LLaMA model with memory optimization
pipe = pipeline(
    "text-generation",
    model=model,  
    tokenizer = tokenizer,
    torch_dtype=torch.float16, # Use FP16 to reduce memory
     device_map="auto"  # Automatically allocate to available GPU/CPU
)

Device set to use cuda:0


In [4]:
# Load bios
bios_df = pd.read_csv("accountant_samples.csv")

In [5]:
import os

# Load job ads from individual files
job_ads_dir = "job_ads/jobADs_len2048"
job_ads_files = sorted([f for f in os.listdir(job_ads_dir) if f.endswith(".txt")])

job_ads = []
for filename in job_ads_files:
    with open(os.path.join(job_ads_dir, filename), "r", encoding="utf-8") as f:
        job_ads.append(f.read().strip())

In [9]:
# --- Prompt Templates ---
with open("prompts/prompt_CV.txt", "r", encoding="utf-8") as f:
    cv_template = f.read()

with open("prompts/prompt_CoverLetter.txt", "r", encoding="utf-8") as f:
    cover_letter_template = f.read()

In [10]:
def format_prompt(template, bio, job_ad):
    return f"<s>[INST] {template.format(bio=bio, job_ad=job_ad)} [/INST]"

In [12]:

# Create output directory
os.makedirs("gen_CV_CL_len2048", exist_ok=True)

In [14]:

# --- Generate Outputs ---

for i, (index, row) in enumerate(bios_df.iterrows()):
    bio = row["hard_text"] 
    profession = row.get("profession", "N/A")
    gender = row.get("gender", "N/A")
    job_ad = job_ads[i]  # 1-to-1 mapping

    
  
    cv_prompt = format_prompt(cv_template, bio, job_ad)
    cover_prompt = format_prompt(cover_letter_template, bio, job_ad)
        
    cv_response = pipe(cv_prompt, max_new_tokens=2048, temperature=0.0, do_sample=False)[0]["generated_text"]
    cover_response = pipe(cover_prompt, max_new_tokens=2048, temperature=0.0, do_sample=False)[0]["generated_text"]
    
        
    print(f"\n--- CV and Cover Letter for Bio #{i+1}, Job Ad #{i+1} ---")
    print(f"\nCV:\n{cv_response}\n")
    print(f"Cover Letter:\n{cover_response}\n")

# Save each CV and Cover Letter in separate files
    with open(f"gen_CV_CL_len2048/cv_{i+1}.txt", "w", encoding="utf-8") as f:
        f.write(cv_response)

    with open(f"gen_CV_CL_len2048/cover_letter_{i+1}.txt", "w", encoding="utf-8") as f:
        f.write(cover_response)

    print(f"Saved CV and Cover Letter for Bio #{i+1}")


--- CV and Cover Letter for Bio #1, Job Ad #1 ---

CV:
<s>[INST] "Generate a professional CV based on the following biography:\n\nShe was raised up in a remote rural area where most of the basic needs, education materials, and spiritual guidance were offered by charity organizations. This grew her deep appreciation for non-profits and she was inspired from her early childhood years to take part in empowering people from humble backgrounds. Lucy joined the SAI/Servant Forge team in 2015 as an accounts intern and acquired skills that she now applies to serve as accounts and administration assistant. She is thankful that SAI/ Servant Forge has given her an opportunity to fulfill her dream of serving the community.\n\n"

Generated CV:


 [/INST]  Sure, here is a professional CV based on the biography provided:

Lucy Johnson

Contact Information:

Email: [lucyjohnson@email.com](mailto:lucyjohnson@email.com)
Phone: 555-555-5555

Summary:
Dedicated and passionate professional with experience

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset



--- CV and Cover Letter for Bio #4, Job Ad #4 ---

CV:
<s>[INST] "Generate a professional CV based on the following biography:\n\nIn addition, Jaime holds a bachelor&apos;s degree in accounting and a master&apos;s degree in accounting and information management. She is committed to a holistic approach to educate and empower others to attain their personal, business and financial goals.\n\n"

Generated CV:


 [/INST]  Sure, here is a sample professional CV based on the biography provided:

Jaime Smith

Contact Information:

* Email: [jsmith@email.com](mailto:jsmith@email.com)
* Phone: 555-555-5555
* LinkedIn: linkedin.com/in/jaimesmith

Summary:
Highly motivated and experienced accounting professional with a strong educational background in accounting and information management. Committed to providing holistic and empowering education and support to individuals and businesses to attain their personal, business, and financial goals.

Professional Experience:

Accountant, XYZ Company (20